# Εργαστήριο 12
Όπως και στα προηγούμενα εργαστήρια συνεχίζουμε στο περιβάλλον του [online chisel bootcamp](https://mybinder.org/v2/gh/freechipsproject/chisel-bootcamp/master).

Πριν ξεκινήσετε, εκτελέστε τα επόμενα 2 κελιά:

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## Προαπαιτούμενα
Για το σημερινό εργαστήριο θα χρειαστείτε:
* Το module `DataPath` (εργαστήριο 9) και τα submodules `Alu` και `Registerfile`.
* Το module `FetchUnit` (εργαστήριο 11) και τα submodules `InstructionMemory` και `PcLogic`.

Αντιγράψτε το κώδικά σας για τα modules αυτά στο επόμενο κελί:

In [ ]:
class RegisterFile(word_w: Int, sel_w: Int) extends Module {

    // ...συμπληρώστε...
   
}


class Alu(word_w: Int) extends Module {

    // ...συμπληρώστε...

}


class DataPath(word_w: Int, sel_w: Int) extends Module {

    // ...συμπληρώστε...

}


class InstructionMemory(addr_w: Int, instr_w: Int) extends Module {

    // ...συμπληρώστε...
    
}


class PcLogic(addr_w: Int) extends Module {

    // ...συμπληρώστε...
    
}


class FetchUnit extends Module {
    
    // ...συμπληρώστε...
    
}

## Αποκωδικοποίηση εντολών
Στην απλή ΚΜΕ που σχεδιάζουμε, το κύκλωμα της **αποκωδικοποίησης** των εντολών μηχανής μετατρέπει τα bits των υποστηριζόμενων εντολών στα αντίστοιχα σήματα ελέγχου προς τα κυκλώματα εκτέλεσης των πράξεων και το register file.

Το κύκλωμα αποκωδικοποίησης (module `DecodeUnit`) δέχεται ως είσοδο τα bits κάθε εντολής (`instr_bits`, εύρος 16 bits) και παράγει τα σήματα ελέγχου του module `DataPath`, δηλαδή τα εξής:
* `im`, εύρος 8 bits
* `sel`, εύρος 2 bits
* `sub`, εύρος 1 bit
* `alu_a_sel`, εύρος 1 bit
* `read_sel_a`, εύρος 3 bits
* `read_sel_b`, εύρος 3 bits
* `write_sel`, εύρος 3 bits

*(Για τη λειτουργικότητα των πιο πάνω σημάτων, ανατρέξτε στο εργαστήριο 11.)*

### Άσκηση 1
Υλοποιήστε στο επόμενο κελί το module `DecodeUnit`, το οποίο υλοποιεί τα προηγούμενα.

*Θυμηθείτε ότι όλα τα σήματα ελέγχου που παράγει το `DecodeUnit` πρέπει να έχουν τιμή, ακόμα κι αν μια εντολή δεν χρησιμοποιεί κάποια από αυτά!*

In [ ]:
class DecodeUnit extends Module {
  val io = IO(new Bundle {
    val instr_bits = Input(UInt(16.W))

    val im = Output(UInt(8.W))
    val sel = Output(UInt(2.W))
    val sub = Output(UInt(1.W))
    var alu_a_sel = Output(UInt(1.W))
    val read_sel_a = Output(UInt(3.W))
    val read_sel_b = Output(UInt(3.W))
    val write_sel = Output(UInt(3.W))
  })
  
  // ...συμπληρώστε...
}

Ελέγξτε το κύκλωμά σας με τον παρακάτω δοκιμαστικό κώδικα, ο οποίος χρησιμοποιεί μια εντολή ανάθεσης σταθεράς σε καταχωρητή και μία εντολή πρόσθεσης μεταξύ καταχωρητών:

In [ ]:
test(new DecodeUnit()) { c => 
  c.io.instr_bits.poke("b0001010000000001".U)  // R2 <- 1
  c.io.im.expect(1.U)
  c.io.sel.expect(1.U)
  c.io.alu_a_sel.expect(0.U)
  c.io.read_sel_b.expect(0.U)
  c.io.write_sel.expect(2.U)
    
  c.io.instr_bits.poke("b0000001011001010".U)  // R1 <- R1 + R2
  c.io.sel.expect(3.U)
  c.io.alu_a_sel.expect(1.U)
  c.io.read_sel_a.expect(1.U)
  c.io.read_sel_b.expect(2.U)
  c.io.write_sel.expect(1.U)
}
println("SUCCESS!!")

## Η τελική ΚΜΕ
Το module `Cpu` συνδυάζει τα `FetchUnit`, `DecodeUnit`, `Datapath` για τη συγκρότηση της τελικής ΚΜΕ.

Συμπληρώστε το επόμενο κελί σύμφωνα με τις εξής οδηγίες:

1. Οδηγήστε τις εισόδους `offset` και `pc_sel` του `fetch_unit` στο 0 (η ΚΜΕ στην τρέχουσα μορφή της δεν υποστηρίζει εντολές διακλάδωσης).

2. Οδηγήστε την έξοδο `instr_bits` του `fetch_unit` στην αντίστοιχη είσοδο του `decode_unit`.

3. Οδηγήστε τα σήματα `im`, `sel, `sub`, `alu_a_sel`, `read_sel_a`, `read_sel_b` και `write_sel` του `decode_unit` στις αντίστοιχες εισόδους του `datapath`.

4. Οδηγήστε την έξοδο `results` του `datapath` στην έξοδο `results`.

In [ ]:
class Cpu extends Module {
  val io = IO(new Bundle {
    val results = Output(UInt(8.W))  
  })
    
  val fetch_unit = Module(new FetchUnit())
  val decode_unit = Module(new DecodeUnit())
  val datapath = Module(new DataPath(8,3))

  // ...συμπληρώστε...  
}

Δοκιμάστε το κυκλωμά σας. Αν η μνήμη εντολών περιέχει τις εντολές μηχανής του εργαστηρίου 11, με τον πιο κάτω κώδικα θα πρέπει να δείτε στην έξοδο `results` τα αποτελέσματα της εκτέλεσης των εντολών αυτών. 

In [ ]:
test(new Cpu()) { c =>
  println(c.io.results.peek())
  for (i <- 0 to 9) {
    c.clock.step()
    println(c.io.results.peek())
  }
}

## Ανεβάστε τη λύση σας στο opencourses
Ανεβάστε το τελικό σας notebook στο opencourses (**Εργασία 3**) έως και τη **Δευτέρα 24/1**.

*(Ίσως χρειαστεί να κάνετε zip το αρχείο ipynb για να γίνει αποδεκτό από το σύστημα)*

## Τελειώνοντας
Έχοντας ολοκληρώσει το εργαστήριο, έχετε σχεδιάσει μια μικρή ΚΜΕ η οποία υποστηρίζει όλα τα στάδια του κύκλου μηχανής (ανάκληση, αποκωδικοποίηση, εκτέλεση, αποθήκευση) **εκτός** από την αποθήκευση δεδομένων στη μνήμη (δηλ. δεν υπάρχει μνήμη δεδομένων).

Επίσης δεν υποστηρίζονται διακλαδώσεις, για τις οποίες πρέπει να υλοποιηθούν πρόσθετες εντολές.

